In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(data.table)
library(dplyr); library(tidyr)
library(magrittr)
library(picante)
library(ape)

data.table 1.9.4  For help type: ?data.table
*** NB: by=.EACHI is now explicit. See README to restore previous behaviour.

Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, last

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘magrittr’

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: ape
Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.2-1
Loading required package: nlme

Attaching package: ‘nlme’

The following object is masked from ‘package:dplyr’:

    collapse



In [3]:
%%R
df.l2fc = read.csv("../data/l2fc_table.csv")

In [4]:
%%R
otus = df.l2fc %>% select(OTU) %>% extract2(1) %>% unique
tree = read.tree("../data/otusn.tree") 
tree = drop.tip(tree, tree$tip.label[-match(otus, tree$tip.label)])
write.tree(tree, "../data/tmp/passed_sparsity.tree")
tree


Phylogenetic tree with 1048 tips and 1047 internal nodes.

Tip labels:
	OTU.783, OTU.759, OTU.2638, OTU.79, OTU.696, OTU.1011, ...
Node labels:
	0.701.5, 0.959.22, 0.594.2, 0.801.3, 0.708.2, 0.940, ...

Rooted; includes branch lengths.


In [34]:
%%R
comm = df.l2fc %>%
    filter(padj <= 0.10) %>%
    group_by(OTU, Treatment) %>%
    filter(row_number(OTU) == 1) %>%
    mutate(N = 1) %>%
    select(Treatment, OTU, N) %>%
    group_by() %>%
    spread(OTU, N, fill = 0)

rownames(comm) = comm$Treatment

comm$Treatment = NULL

t = drop.tip(tree, tree$tip.label[-match(colnames(comm), tree$tip.label)])

phy.dist = cophenetic(t)

ses.mpd(comm, phy.dist, null.model = "independentswap", abundance.weighted = FALSE, 
    runs = 999) %>% 
    add_rownames("Treatment") %>%
    select(Treatment, ntaxa, mpd.obs.z, mpd.obs.p) %>%
    mutate(NRI = mpd.obs.z * -1) %>% print

ses.mntd(comm, phy.dist, null.model = "independentswap", abundance.weighted = FALSE, 
    runs = 999) %>%
    add_rownames("Treatment") %>%
    select(Treatment, ntaxa, mntd.obs.z, mntd.obs.p) %>%
    mutate(NTI = mntd.obs.z * -1) %>% print

  Treatment ntaxa mpd.obs.z mpd.obs.p       NRI
1    13CCPS    63 -4.626698     0.001  4.626698
2    13CXPS    49  1.406115     0.928 -1.406115
  Treatment ntaxa mntd.obs.z mntd.obs.p      NTI
1    13CCPS    63  -1.338352      0.090 1.338352
2    13CXPS    49  -2.780160      0.002 2.780160


In [88]:
%%R
library(adephylo)

table = df.l2fc %>%
    group_by(OTU, Treatment) %>%
    filter(rank(padj, ties.method = "first") == 1) %>%
    mutate(N = ifelse(padj <= 0.10, 1, 0)) %>%
    select(Treatment, OTU, N) %>%
    group_by() %>%
    spread(Treatment, N, fill = 0)

otus = df.l2fc %>% select(OTU) %>% extract2(1) %>% unique
tree_all = read.tree("../data/otusn.tree") 
tree_all = drop.tip(tree_all, tree_all$tip.label[-match(otus, tree_all$tip.label)])
m = 1

#######################################################################
#
#   THE FOLLOWING IS FROM CONSENTRAIT WEBSITE!!
#
#######################################################################

#Starting script
Mean_all<-matrix(nrow=ncol(table)-1,ncol=1)

#for (m in 1:length(tree_all)) {#loop through all trees

  tree<-tree_all #[[m]]
# testing if table and tree contain the same entries - else drop tips
  z<-subset(tree$tip.label,!(tree$tip.label %in% table[,1]))
  if (length(z) > 0) {
    drop.tip(tree,z)
  }

#rooting tree with first taxon - change if different root
  root_tree<-root(unroot(tree),1,resolve.root=T)
#replacing negative branch lengths - e.g., from PHYLIP
  root_tree$edge.length[root_tree$edge.length <= 0] =  0.00001
  subtree<-subtrees(root_tree, wait=FALSE)

  cluster_mean<-numeric(length=0)

# loop through all traits
  for (j in 2:ncol(table)) {
     print(c("Analyzing",j,"!!!!!!!!!!!!!!!!!!!!!!!"))
     #Loading trait table
     table_tmp<-table[,c(1,j)]
     colnames(table_tmp)[1]<-"ID";
     colnames(table_tmp)[2]<-"Trait";
     
# removing all entries not in tree 
    table_tmp2<-data.table(table_tmp)
     setkey(table_tmp2,ID)
     table2<-table_tmp2[intersect(table_tmp2$ID,root_tree$tip.label)]
     setkey(table2,ID)

     #initializing result vectors and file names
     positives<-vector(mode="list",length=0)
     cluster_size<-numeric(length=0)
     cluster_size_file<-paste("R_cluster_size_",j,".txt",sep="")

     cluster_dist<-numeric(length=0)
     cluster_dist_file<-paste("R_cluster_dist_",j,".txt",sep="")

     #loop through all subtrees and determining if any subtrees have >90% positives
     for (i in 1:length(subtree)){
       tip_names<-subtree[[i]]$tip.label
       if (mean(table2[tip_names][,Trait]) > 0.9 ) {#change this value if you want a new threshold
        match_test<-match(tip_names,positives)
        if (all(is.na(match_test))) {
            positives<-c(positives,tip_names)
            cluster_dist<-distRoot(subtree[[i]],tip_names, method=c("p"))
            cluster_size<-c(cluster_size,mean(cluster_dist))

            # printing to files###
            cat(c(j,m,mean(cluster_dist),length(cluster_dist)), file = cluster_size_file, sep = "\t", fill = FALSE, labels = NULL,append = TRUE)
            cat("\n", file = cluster_size_file, fill = FALSE, labels = NULL,append = TRUE)

            cat(j,m,cluster_dist, file = cluster_dist_file, sep = "\t", fill = FALSE, labels = NULL,append = TRUE)
            cat("\n", file = cluster_dist_file, fill = FALSE, labels = NULL,append = TRUE)


            #print(cluster_dist)
        }
        else if (any(is.na(match_test))) {
            print("some NAs - something is weird")
        }
        else {
            #print(tip_names)
            #print("found cluster before")
        }
      }
    }

    ##### find singletons ######
    a<-table2[table2$Trait == 1,][,ID]
    g<-as.character(a)

    singletons_names = setdiff(g,positives)
    if (length(singletons_names) > 0) {
       for (h in 1:length(singletons_names)){
           # weigh singletons with half
           singleton_edges = 0.5*root_tree$edge.length[which.edge(root_tree,singletons_names[h])] #here we use half the distance for singletons
           cluster_size<-c(cluster_size,singleton_edges)

           cat(c(j,m,singleton_edges,1), file = cluster_size_file, sep = "\t", fill = FALSE, labels = NULL,append = TRUE)
           cat("\n", file = cluster_size_file, sep = "\t", fill = FALSE, labels = NULL,append = TRUE)
       }
    
    }
  Mean_all[j-1,m] = mean(cluster_size)
  }
  
#}
#output file
#write.table(Mean_all,"Mean_all_bootstrap2.txt", sep = "\t")
Mean_all

[1] "Analyzing"               "2"                      
[3] "!!!!!!!!!!!!!!!!!!!!!!!"
[1] "Analyzing"               "3"                      
[3] "!!!!!!!!!!!!!!!!!!!!!!!"
           [,1]
[1,] 0.02877491
[2,] 0.01213100


In [47]:
%%R
df.l2fc %>%
    group_by(OTU, Treatment, Rank2) %>%
    filter(rank(padj, ties.method = "first") == 1) %>%
    mutate(N = ifelse(padj <= 0.10, 1, 0)) %>%
    group_by() %>%
    mutate(cmbnd = paste(Treatment, Rank2, sep = "_")) %>%
    group_by(cmbnd) %>%
    mutate(total = sum(N)) %>%
    filter(total >= 10) %>%
    select(cmbnd, OTU, N) %>%
    spread(cmbnd, N, fill = 0)

Source: local data frame [605 x 7]

        OTU 13CCPS_Planctomycetes 13CCPS_Proteobacteria 13CCPS_Verrucomicrobia
1    OTU.10                     0                     0                      0
2   OTU.100                     0                     1                      0
3  OTU.1001                     0                     0                      0
4  OTU.1004                     0                     0                      0
5   OTU.101                     0                     0                      0
6  OTU.1011                     0                     0                      0
7  OTU.1012                     0                     0                      0
8  OTU.1016                     0                     0                      0
9  OTU.1018                     0                     0                      0
10  OTU.102                     0                     0                      0
..      ...                   ...                   ...                    ...
Variables not sh